In [15]:
import tensorflow as tf
tf.enable_eager_execution()

In [16]:

chinese_embedding = tf.constant([[0.11, 0.21,0.31,0.41],
                                 [0.21,0.31,0.41,0.51],
                                 [0.31,0.41,0.51,0.61],
                                 [0.41,0.51,0.61,0.71]], dtype=tf.float32)

position_embedding = tf.constant([[0.01,0.01,0.01,0.01],
                                  [0.02,0.02,0.02,0.02],
                                  [0.03,0.03,0.03,0.03],
                                  [0.04,0.04,0.04,0.04]], dtype=tf.float32)

encoder_input = tf.constant([[0,1,2,3],[2,3,0,1]], dtype=tf.int32)

with tf.variable_scope('encoder_input'):
    encoder_embedding_input = tf.nn.embedding_lookup(chinese_embedding, encoder_input)
    encoder_embedding_input += position_embedding
    
print('encoder_embedding_input:')
print(encoder_embedding_input)


encoder_embedding_input:
tf.Tensor(
[[[0.12       0.22       0.32       0.42      ]
  [0.22999999 0.33       0.43       0.53      ]
  [0.34       0.44       0.53999996 0.64      ]
  [0.45       0.55       0.65000004 0.75      ]]

 [[0.32       0.42       0.52       0.62      ]
  [0.43       0.53       0.63       0.72999996]
  [0.14       0.24       0.34       0.44      ]
  [0.25       0.35       0.45       0.55      ]]], shape=(2, 4, 4), dtype=float32)


In [17]:
encoder_w_Q = tf.constant([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
                           [0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
                           [0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
                           [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]], dtype=tf.float32)

encoder_w_K = tf.constant([[0.08, 0.18, 0.28, 0.38, 0.48, 0.58],
                           [0.18, 0.28, 0.38, 0.48, 0.58, 0.68],
                           [0.28, 0.38, 0.48, 0.58, 0.68, 0.78],
                           [0.38, 0.48, 0.58, 0.68, 0.78, 0.88]], dtype=tf.float32)

encoder_w_V = tf.constant([[0.12, 0.22, 0.32, 0.42, 0.52, 0.62],
                           [0.22, 0.32, 0.42, 0.52, 0.62, 0.72],
                           [0.32, 0.42, 0.52, 0.62, 0.72, 0.82],
                           [0.42, 0.52, 0.62, 0.72, 0.82, 0.92]], dtype=tf.float32)

with tf.variable_scope('encoder_scaled_dot_product_attention'):
    encoder_Q = tf.matmul(tf.reshape(encoder_embedding_input, (-1, tf.shape(encoder_embedding_input)[2])), encoder_w_Q)
    encoder_K = tf.matmul(tf.reshape(encoder_embedding_input, (-1, tf.shape(encoder_embedding_input)[2])), encoder_w_K)
    encoder_V = tf.matmul(tf.reshape(encoder_embedding_input, (-1, tf.shape(encoder_embedding_input)[2])), encoder_w_V)
    
    print('encoder_Q:')
    print(encoder_Q)
    print('encoder_K:')
    print(encoder_K)
    print('encoder_V:')
    print(encoder_V)
    
    encoder_Q = tf.reshape(encoder_Q, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    encoder_K = tf.reshape(encoder_K, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    encoder_V = tf.reshape(encoder_V, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    
    print()
    print('after reshape:')
    print('encoder_Q:')
    print(encoder_Q)
    print('encoder_K:')
    print(encoder_K)
    print('encoder_V:')
    print(encoder_V)
    
    attention_map = tf.matmul(encoder_Q, tf.transpose(encoder_K, [0, 2, 1]))
    attention_map /= 8
    attention_map = tf.nn.softmax(attention_map)
    
    print()
    print('attention_map:')
    print(attention_map)
    
    encoder_first_as_output = tf.matmul(attention_map, encoder_V)
    
    print()
    print('encoder_first_as_output:')
    print(encoder_first_as_output)
    

encoder_Q:
tf.Tensor(
[[0.32       0.428      0.536      0.644      0.752      0.85999995]
 [0.43       0.582      0.734      0.886      1.038      1.1899999 ]
 [0.54       0.73599994 0.932      1.128      1.324      1.52      ]
 [0.65000004 0.89       1.1300001  1.37       1.61       1.85      ]
 [0.52       0.708      0.896      1.084      1.272      1.46      ]
 [0.63       0.862      1.094      1.326      1.558      1.79      ]
 [0.34       0.456      0.572      0.68799996 0.804      0.91999996]
 [0.45000002 0.61       0.77000004 0.93       1.09       1.25      ]], shape=(8, 6), dtype=float32)
encoder_K:
tf.Tensor(
[[0.29839998 0.4064     0.5144     0.6224     0.73039997 0.83839995]
 [0.3996     0.5516     0.7036     0.8556     1.0076     1.1596    ]
 [0.5008     0.6968     0.8927999  1.0888     1.2847999  1.4807999 ]
 [0.602      0.842      1.082      1.322      1.562      1.8019999 ]
 [0.4824     0.67039996 0.8584     1.0464     1.2343999  1.4224    ]
 [0.5836     0.8156     1.04

In [25]:
print('Multi-Head Attention:')
w_Z = tf.constant([[0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4]], dtype=tf.float32)

with tf.variable_scope('encoder_input'):
    encoder_embedding_input = tf.nn.embedding_lookup(chinese_embedding, encoder_input)
    encoder_embedding_input += position_embedding
    
with tf.variable_scope('encoder_multi_head_product_attention'):
    encoder_Q = tf.matmul(tf.reshape(encoder_embedding_input, (-1, tf.shape(encoder_embedding_input)[2])), encoder_w_Q)
    encoder_K = tf.matmul(tf.reshape(encoder_embedding_input, (-1, tf.shape(encoder_embedding_input)[2])), encoder_w_K)
    encoder_V = tf.matmul(tf.reshape(encoder_embedding_input, (-1, tf.shape(encoder_embedding_input)[2])), encoder_w_V)
    
    print('encoder_Q:')
    print(encoder_Q)
    print('encoder_K:')
    print(encoder_K)
    print('encoder_V:')
    print(encoder_V)
    
    encoder_Q = tf.reshape(encoder_Q, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    encoder_K = tf.reshape(encoder_K, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    encoder_V = tf.reshape(encoder_V, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    
    print()
    print('after reshape:')
    print('encoder_Q:')
    print(encoder_Q)
    print('encoder_K:')
    print(encoder_K)
    print('encoder_V:')
    print(encoder_V)
    
    encoder_Q_split = tf.split(encoder_Q, 2, axis=2)
    encoder_K_split = tf.split(encoder_K, 2, axis=2)
    encoder_V_split = tf.split(encoder_V, 2, axis=2)
    
    print('encoder_Q_split:')
    print(encoder_Q_split)
    print('encoder_K_split:')
    print(encoder_K_split)
    print('encoder_V_split:')
    print(encoder_V_split)
    
    encoder_Q_concat = tf.concat(encoder_Q_split, axis=0)
    encoder_K_concat = tf.concat(encoder_K_split, axis=0)
    encoder_V_concat = tf.concat(encoder_V_split, axis=0)
    
    print('encoder_Q_concat:')
    print(encoder_Q_concat)
    print('encoder_K_concat:')
    print(encoder_K_concat)
    print('encoder_V_concat:')
    print(encoder_V_concat)
    
    attention_map = tf.matmul(encoder_Q_concat, tf.transpose(encoder_K_concat, [0, 2, 1]))
    attention_map /= 8
    attention_map = tf.nn.softmax(attention_map)
    
    print('attention_map')
    print(attention_map)
    
    weightedSumV = tf.matmul(attention_map, encoder_V_concat)
    
    print('weightedSumV:')
    print(weightedSumV)
    
    outputs_z = tf.concat(tf.split(weightedSumV, 2, axis=0), axis=2)
    
    print('outputs_z:')
    print(outputs_z)
    
    outputs = tf.matmul(tf.reshape(outputs_z, (-1, tf.shape(outputs_z)[2])), w_Z)
    outputs = tf.reshape(outputs, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    
    print('outputs:')
    print(outputs)
    

Multi-Head Attention:
encoder_Q:
tf.Tensor(
[[0.32       0.428      0.536      0.644      0.752      0.85999995]
 [0.43       0.582      0.734      0.886      1.038      1.1899999 ]
 [0.54       0.73599994 0.932      1.128      1.324      1.52      ]
 [0.65000004 0.89       1.1300001  1.37       1.61       1.85      ]
 [0.52       0.708      0.896      1.084      1.272      1.46      ]
 [0.63       0.862      1.094      1.326      1.558      1.79      ]
 [0.34       0.456      0.572      0.68799996 0.804      0.91999996]
 [0.45000002 0.61       0.77000004 0.93       1.09       1.25      ]], shape=(8, 6), dtype=float32)
encoder_K:
tf.Tensor(
[[0.29839998 0.4064     0.5144     0.6224     0.73039997 0.83839995]
 [0.3996     0.5516     0.7036     0.8556     1.0076     1.1596    ]
 [0.5008     0.6968     0.8927999  1.0888     1.2847999  1.4807999 ]
 [0.602      0.842      1.082      1.322      1.562      1.8019999 ]
 [0.4824     0.67039996 0.8584     1.0464     1.2343999  1.4224    ]
 [0.58

In [26]:

with tf.variable_scope('encoder_block'):
    encoder_Q = tf.matmul(tf.reshape(encoder_embedding_input, (-1, tf.shape(encoder_embedding_input)[2])), encoder_w_Q)
    encoder_K = tf.matmul(tf.reshape(encoder_embedding_input, (-1, tf.shape(encoder_embedding_input)[2])), encoder_w_K)
    encoder_V = tf.matmul(tf.reshape(encoder_embedding_input, (-1, tf.shape(encoder_embedding_input)[2])), encoder_w_V)
    
    print('encoder_Q:')
    print(encoder_Q)
    print('encoder_K:')
    print(encoder_K)
    print('encoder_V:')
    print(encoder_V)
    
    encoder_Q = tf.reshape(encoder_Q, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    encoder_K = tf.reshape(encoder_K, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    encoder_V = tf.reshape(encoder_V, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    
    print()
    print('after reshape:')
    print('encoder_Q:')
    print(encoder_Q)
    print('encoder_K:')
    print(encoder_K)
    print('encoder_V:')
    print(encoder_V)
    
    encoder_Q_split = tf.split(encoder_Q, 2, axis=2)
    encoder_K_split = tf.split(encoder_K, 2, axis=2)
    encoder_V_split = tf.split(encoder_V, 2, axis=2)
    
    print('encoder_Q_split:')
    print(encoder_Q_split)
    print('encoder_K_split:')
    print(encoder_K_split)
    print('encoder_V_split:')
    print(encoder_V_split)
    
    encoder_Q_concat = tf.concat(encoder_Q_split, axis=0)
    encoder_K_concat = tf.concat(encoder_K_split, axis=0)
    encoder_V_concat = tf.concat(encoder_V_split, axis=0)
    
    print('encoder_Q_concat:')
    print(encoder_Q_concat)
    print('encoder_K_concat:')
    print(encoder_K_concat)
    print('encoder_V_concat:')
    print(encoder_V_concat)
    
    attention_map = tf.matmul(encoder_Q_concat, tf.transpose(encoder_K_concat, [0, 2, 1]))
    attention_map /= 8
    attention_map = tf.nn.softmax(attention_map)
    
    print('attention_map')
    print(attention_map)
    
    weightedSumV = tf.matmul(attention_map, encoder_V_concat)
    
    print('weightedSumV:')
    print(weightedSumV)
    
    outputs_z = tf.concat(tf.split(weightedSumV, 2, axis=0), axis=2)
    
    print('outputs_z:')
    print(outputs_z)
    
    outputs = tf.matmul(tf.reshape(outputs_z, (-1, tf.shape(outputs_z)[2])), w_Z)
    outputs = tf.reshape(outputs, (tf.shape(encoder_embedding_input)[0], tf.shape(encoder_embedding_input)[1], -1))
    
    print('outputs:')
    print(outputs)
    
    outputs += encoder_embedding_input
    
    print()
    print('add outputs:')
    print(outputs)
    
    W_f = tf.constant([[0.2,0.3,0.5,0.4],
                       [0.2,0.3,0.5,0.4],
                       [0.2,0.3,0.5,0.4],
                       [0.2,0.3,0.5,0.4]])
    
    ffn_outputs = tf.matmul(tf.reshape(outputs, (-1, tf.shape(outputs)[2])), W_f)
    ffn_outputs = tf.reshape(ffn_outputs, (tf.shape(outputs)[0], tf.shape(outputs)[1], -1))
    
    encoder_outputs = ffn_outputs + outputs
    
    print('add encoder_outputs:')
    print(encoder_outputs)
    

encoder_Q:
tf.Tensor(
[[0.32       0.428      0.536      0.644      0.752      0.85999995]
 [0.43       0.582      0.734      0.886      1.038      1.1899999 ]
 [0.54       0.73599994 0.932      1.128      1.324      1.52      ]
 [0.65000004 0.89       1.1300001  1.37       1.61       1.85      ]
 [0.52       0.708      0.896      1.084      1.272      1.46      ]
 [0.63       0.862      1.094      1.326      1.558      1.79      ]
 [0.34       0.456      0.572      0.68799996 0.804      0.91999996]
 [0.45000002 0.61       0.77000004 0.93       1.09       1.25      ]], shape=(8, 6), dtype=float32)
encoder_K:
tf.Tensor(
[[0.29839998 0.4064     0.5144     0.6224     0.73039997 0.83839995]
 [0.3996     0.5516     0.7036     0.8556     1.0076     1.1596    ]
 [0.5008     0.6968     0.8927999  1.0888     1.2847999  1.4807999 ]
 [0.602      0.842      1.082      1.322      1.562      1.8019999 ]
 [0.4824     0.67039996 0.8584     1.0464     1.2343999  1.4224    ]
 [0.5836     0.8156     1.04

In [57]:

print('Decoder Block:')

english_embedding = tf.constant([[0.51,0.61,0.71,0.81],
                         [0.61,0.71,0.81,0.91],
                         [0.71,0.81,0.91,1.01],
                         [0.81,0.91,1.01,1.11]],dtype=tf.float32)

decoder_input = tf.constant([[1, 2], [2, 1]], dtype=tf.int32)

with tf.variable_scope('decoder_input'):
    decoder_embedding_input = tf.nn.embedding_lookup(english_embedding, decoder_input)
    print(decoder_embedding_input)
    decoder_embedding_input += position_embedding[0:tf.shape(decoder_embedding_input)[1]]
    
    print('decoder_embedding_input:')
    print(decoder_embedding_input)
    

Decoder Block:
tf.Tensor(
[[[0.61 0.71 0.81 0.91]
  [0.71 0.81 0.91 1.01]]

 [[0.71 0.81 0.91 1.01]
  [0.61 0.71 0.81 0.91]]], shape=(2, 2, 4), dtype=float32)
decoder_embedding_input:
tf.Tensor(
[[[0.62       0.71999997 0.82       0.92      ]
  [0.72999996 0.83       0.93       1.03      ]]

 [[0.71999997 0.82       0.92       1.02      ]
  [0.63       0.72999996 0.83       0.93      ]]], shape=(2, 2, 4), dtype=float32)


In [60]:

print('decoder self-attention:')

w_Q_decoder_sa = tf.constant([[0.15,0.25,0.35,0.45,0.55,0.65],
                              [0.25,0.35,0.45,0.55,0.65,0.75],
                              [0.35,0.45,0.55,0.65,0.75,0.85],
                              [0.45,0.55,0.65,0.75,0.85,0.95]], dtype=tf.float32)

w_K_decoder_sa = tf.constant([[0.13,0.23,0.33,0.43,0.53,0.63],
                              [0.23,0.33,0.43,0.53,0.63,0.73],
                              [0.33,0.43,0.53,0.63,0.73,0.83],
                              [0.43,0.53,0.63,0.73,0.83,0.93]], dtype=tf.float32)

w_V_decoder_sa = tf.constant([[0.17,0.27,0.37,0.47,0.57,0.67],
                              [0.27,0.37,0.47,0.57,0.67,0.77],
                              [0.37,0.47,0.57,0.67,0.77,0.87],
                              [0.47,0.57,0.67,0.77,0.87,0.97]], dtype=tf.float32)

with tf.variable_scope("decoder_sa_block"):
    decoder_Q = tf.matmul(tf.reshape(decoder_embedding_input,(-1, tf.shape(decoder_embedding_input)[2])), w_Q_decoder_sa)
    decoder_K = tf.matmul(tf.reshape(decoder_embedding_input,(-1, tf.shape(decoder_embedding_input)[2])), w_K_decoder_sa)
    decoder_V = tf.matmul(tf.reshape(decoder_embedding_input,(-1, tf.shape(decoder_embedding_input)[2])), w_V_decoder_sa)
    
    decoder_Q = tf.reshape(decoder_Q, (tf.shape(decoder_embedding_input)[0], tf.shape(decoder_embedding_input)[1], -1))
    decoder_K = tf.reshape(decoder_K, (tf.shape(decoder_embedding_input)[0], tf.shape(decoder_embedding_input)[1], -1))
    decoder_V = tf.reshape(decoder_V, (tf.shape(decoder_embedding_input)[0], tf.shape(decoder_embedding_input)[1], -1))
          
    decoder_Q_split = tf.split(decoder_Q,2,axis=2)
    decoder_K_split = tf.split(decoder_K,2,axis=2)
    decoder_V_split = tf.split(decoder_V,2,axis=2)
    
    decoder_Q_concat = tf.concat(decoder_Q_split,axis=0)
    decoder_K_concat = tf.concat(decoder_K_split,axis=0)
    decoder_V_concat = tf.concat(decoder_V_split,axis=0)
    
    decoder_sa_attention_map_raw = tf.matmul(decoder_Q_concat, tf.transpose(decoder_K_concat,[0,2,1]))
    decoder_sa_attention_map = decoder_sa_attention_map_raw / 8
    
    '''
        mask的作用是:
            在decoder过程中,得到每个阶段的输出的时候,我们是看不到后面的信息的,
            for example:
                机 器 学 习 ---> machine learning
                在输入machine的时候,无法看到learning的信息,因此在计算attention矩阵的时候[machine, learning]
                的信息是没有的,所以要mask掉
    ''' 
    print('before mask:', decoder_sa_attention_map)
    
    diag_vals = tf.ones_like(decoder_sa_attention_map[0, :, :])
    print('diag_vals', diag_vals)
    
    tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense()
    print('tril', tril)
    
    masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(decoder_sa_attention_map)[0], 1, 1])
    print('masks', masks)
    
    paddings = tf.ones_like(masks) * (-2 ** 32 + 1)
    print('paddings', paddings)
    
    decoder_sa_attention_map = tf.where(tf.equal(masks, 0), paddings, decoder_sa_attention_map)
    print('decoder_sa_attention_map', decoder_sa_attention_map)
    
    decoder_sa_attention_map = tf.nn.softmax(decoder_sa_attention_map)

    print('decoder_sa_attention_map:')
    print(decoder_sa_attention_map)
    

decoder self-attention:
before mask: tf.Tensor(
[[[0.61042327 0.69419265]
  [0.69435763 0.78964627]]

 [[0.7724013  0.6952947 ]
  [0.6951597  0.62576425]]

 [[1.7976708  2.0485718 ]
  [2.048737   2.3346798 ]]

 [[2.2829647  2.051618  ]
  [2.0514832  1.8435937 ]]], shape=(4, 2, 2), dtype=float32)
diag_vals tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
tril tf.Tensor(
[[1. 0.]
 [1. 1.]], shape=(2, 2), dtype=float32)
masks tf.Tensor(
[[[1. 0.]
  [1. 1.]]

 [[1. 0.]
  [1. 1.]]

 [[1. 0.]
  [1. 1.]]

 [[1. 0.]
  [1. 1.]]], shape=(4, 2, 2), dtype=float32)
paddings tf.Tensor(
[[[-4.2949673e+09 -4.2949673e+09]
  [-4.2949673e+09 -4.2949673e+09]]

 [[-4.2949673e+09 -4.2949673e+09]
  [-4.2949673e+09 -4.2949673e+09]]

 [[-4.2949673e+09 -4.2949673e+09]
  [-4.2949673e+09 -4.2949673e+09]]

 [[-4.2949673e+09 -4.2949673e+09]
  [-4.2949673e+09 -4.2949673e+09]]], shape=(4, 2, 2), dtype=float32)
decoder_sa_attention_map tf.Tensor(
[[[ 6.1042327e-01 -4.2949673e+09]
  [ 6.9435763e-01  7.896462

In [64]:

w_Z_decoder_sa = tf.constant([[0.1,0.2,0.3,0.4],
                              [0.1,0.2,0.3,0.4],
                              [0.1,0.2,0.3,0.4],
                              [0.1,0.2,0.3,0.4],
                              [0.1,0.2,0.3,0.4],
                              [0.1,0.2,0.3,0.4]], dtype=tf.float32)

weightedSumV = tf.matmul(decoder_sa_attention_map, decoder_V_concat)
# print(weightedSumV)
decoder_outputs_z = tf.concat(tf.split(weightedSumV, 2, axis=0), axis=2)
# print(decoder_outputs_z)
decoder_sa_outputs = tf.matmul(tf.reshape(decoder_outputs_z, (-1, tf.shape(decoder_outputs_z)[2])), w_Z_decoder_sa)

decoder_sa_outputs = tf.reshape(decoder_sa_outputs, (tf.shape(decoder_embedding_input)[0], tf.shape(decoder_embedding_input)[1], -1))

print('decoder_sa_outputs:')
print(decoder_sa_outputs)


decoder_sa_outputs:
tf.Tensor(
[[[1.0833601 2.1667202 3.25008   4.3334403]
  [1.1666678 2.3333356 3.5000033 4.6666713]]

 [[1.22016   2.44032   3.66048   4.88064  ]
  [1.1634135 2.326827  3.4902406 4.653654 ]]], shape=(2, 2, 4), dtype=float32)


In [69]:

print('encoder-decoder attention:')

w_Q_decoder_sa2 = tf.constant([[0.2,0.3,0.4,0.5,0.6,0.7],
                   [0.3,0.4,0.5,0.6,0.7,0.8],
                   [0.4,0.5,0.6,0.7,0.8,0.9],
                   [0.5,0.6,0.7,0.8,0.9,1]],dtype=tf.float32)

w_K_decoder_sa2 = tf.constant([[0.18,0.28,0.38,0.48,0.58,0.68],
                   [0.28,0.38,0.48,0.58,0.68,0.78],
                   [0.38,0.48,0.58,0.68,0.78,0.88],
                   [0.48,0.58,0.68,0.78,0.88,0.98]],dtype=tf.float32)

w_V_decoder_sa2 = tf.constant([[0.22,0.32,0.42,0.52,0.62,0.72],
                   [0.32,0.42,0.52,0.62,0.72,0.82],
                   [0.42,0.52,0.62,0.72,0.82,0.92],
                   [0.52,0.62,0.72,0.82,0.92,1.02]],dtype=tf.float32)

w_Z_decoder_sa2 = tf.constant([[0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4],
                   [0.1,0.2,0.3,0.4]],dtype=tf.float32)

with tf.variable_scope('decoder_encoder_attention_block:'):
    
    decoder_sa_outputs += decoder_embedding_input
    
    encoder_decoder_Q = tf.matmul(tf.reshape(decoder_sa_outputs, (-1, tf.shape(decoder_sa_outputs)[2])), w_Q_decoder_sa2)
    encoder_decoder_K = tf.matmul(tf.reshape(encoder_outputs, (-1, tf.shape(encoder_outputs)[2])), w_K_decoder_sa2)
    encoder_decoder_V = tf.matmul(tf.reshape(encoder_outputs, (-1, tf.shape(encoder_outputs)[2])), w_V_decoder_sa2)
    
    encoder_decoder_Q = tf.reshape(encoder_decoder_Q, (tf.shape(decoder_embedding_input)[0], tf.shape(decoder_embedding_input)[1], -1))
    encoder_decoder_K = tf.reshape(encoder_decoder_K, (tf.shape(encoder_outputs)[0], tf.shape(encoder_outputs)[1], -1))
    encoder_decoder_V = tf.reshape(encoder_decoder_V, (tf.shape(encoder_outputs)[0], tf.shape(encoder_outputs)[1], -1))
    
    encoder_decoder_Q_split = tf.split(encoder_decoder_Q, 2, axis=2)
    encoder_decoder_K_split = tf.split(encoder_decoder_K, 2, axis=2)
    encoder_decoder_V_split = tf.split(encoder_decoder_V, 2, axis=2)
    
    encoder_decoder_Q_concat = tf.concat(encoder_decoder_Q_split, axis=0)
    encoder_decoder_K_concat = tf.concat(encoder_decoder_K_split, axis=0)
    encoder_decoder_V_concat = tf.concat(encoder_decoder_V_split, axis=0)
    
    encoder_decoder_attention_map_raw = tf.matmul(encoder_decoder_Q_concat, tf.transpose(encoder_decoder_K_concat, [0, 2, 1]))
    encoder_decoder_attention_map_raw /= 8
    encoder_decoder_attention_map = tf.nn.softmax(encoder_decoder_attention_map_raw)
    
    weightedSumV = tf.matmul(encoder_decoder_attention_map, encoder_decoder_V_concat)
    
    encoder_decoder_outputs_z = tf.concat(tf.split(weightedSumV, 2, axis=0), axis=2)
    encoder_decoder_outputs = tf.matmul(tf.reshape(encoder_decoder_outputs_z, (-1, tf.shape(encoder_decoder_outputs_z)[2])), w_Z_decoder_sa2)
    
    encoder_decoder_attention_outputs = tf.reshape(encoder_decoder_outputs, (tf.shape(decoder_embedding_input)[0], tf.shape(decoder_embedding_input)[1], -1))
    
    encoder_decoder_attention_outputs += decoder_sa_outputs
    # print(encoder_decoder_attention_outputs)
    
    W_f = tf.constant([[0.2,0.3,0.5,0.4],
                       [0.2,0.3,0.5,0.4],
                       [0.2,0.3,0.5,0.4],
                       [0.2,0.3,0.5,0.4]])
    
    decoder_ffn_outputs = tf.matmul(tf.reshape(encoder_decoder_attention_outputs, (-1, tf.shape(encoder_decoder_attention_outputs)[2])), W_f)
    decoder_ffn_outputs = tf.reshape(decoder_ffn_outputs, (tf.shape(encoder_decoder_attention_outputs)[0], tf.shape(encoder_decoder_attention_outputs)[1], -1))
    
    decoder_outputs = decoder_ffn_outputs + encoder_decoder_attention_outputs
    
    print('decoder_outputs:')
    print(decoder_outputs)
    

encoder-decoder attention:
decoder_outputs:
tf.Tensor(
[[[32.96538  52.862305 83.22769  82.18768 ]
  [34.217464 54.50911  85.57657  84.316574]]

 [[33.886017 53.9967   84.782715 83.54272 ]
  [32.895317 52.705524 82.950836 81.89084 ]]], shape=(2, 2, 4), dtype=float32)


In [73]:

W_final = tf.constant([[0.2,0.3,0.5,0.4],
                       [0.2,0.3,0.5,0.4],
                       [0.2,0.3,0.5,0.4],
                       [0.2,0.3,0.5,0.4]])

logits = tf.matmul(tf.reshape(decoder_outputs, (-1, tf.shape(decoder_outputs)[2])), W_final)
logits = tf.reshape(logits, (tf.shape(decoder_outputs)[0], tf.shape(decoder_outputs)[1], -1))

logits = tf.nn.softmax(logits)
print(logits)

y = tf.one_hot(decoder_input, depth=4)
print(y)

# loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
# 
# train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)


tf.Tensor(
[[[1.8448510e-33 1.5041991e-22 1.0000000e+00 1.2264673e-11]
  [2.0177409e-34 3.4401375e-23 1.0000000e+00 5.8652909e-12]]

 [[4.1597021e-34 5.5724043e-23 1.0000000e+00 7.4648256e-12]
  [2.3456182e-33 1.7653890e-22 1.0000000e+00 1.3286693e-11]]], shape=(2, 2, 4), dtype=float32)
tf.Tensor(
[[[0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 0. 1. 0.]
  [0. 1. 0. 0.]]], shape=(2, 2, 4), dtype=float32)
